**Создадим словарь с гексамерами**

In [5]:
hexa_dict = {}

**Зададим функцию, которая построит комплементарную цепь** 

Цепь остается неперевернутой

In [6]:
def opposite(x): 
    op_dic = {'G': 'C', 'C': 'G', 'A': 'T', 'T': 'A'}
    ans = ''
    for let in x:
        ans += str(op_dic.get(let)) 
    return ans

**Функция, работающая с гексомерами**

На вход получет:

1) последовательность гена

2) суммарное покрытие гена, нормированное на длину

3) покрытие нуклеотидов в гене

4) информацию о цепи, на которой расположен ген (плюс/минус)

Покрытие гексомеров записывается в словарь

In [7]:
def hexa(seq, s, cov, sign): #функция, которая записывает наши гексомеры в словарь и в соответствие им ставит покрытине нормированное на покрытие гена (нормировано на длину)    
    if sign == '+':
        i = 0
        while i < len(seq) - 5:
        
            hexomer = seq[i: i + 6]
            try:
                p = sum(cov[i + 23: i + 23 + 6])/s
                if hexomer in hexa_dict:
                    hexa_dict.get(hexomer).append(p)
                else:
                    hexa_dict[hexomer] = [p]
            except:
                if hexomer in hexa_dict:
                    hexa_dict.get(hexomer).append(0)
                else:
                    hexa_dict[hexomer] = [0]
            i += 1
    if sign == '-':
        i = len(seq)
        while i > 5:
            hexomer = seq[i- 6: i][::-1]
            try:
                p = sum(cov[i-6-23: i-23])/s
                if hexomer in hexa_dict:
                    hexa_dict.get(hexomer).append(p)
                else:
                    hexa_dict[hexomer] = [p]
            except:
                if hexomer in hexa_dict:
                    hexa_dict.get(hexomer).append(0)
                else:
                    hexa_dict[hexomer] = [0]
            i -= 1


**Основная функция**

На вход получает файлы с репликами плюс и минус цепей

Использует файлы: последовательность генома, координаты генов

Для этих файлов считает покрытие гексомеров с помощью функций выше

In [8]:
def programm(y, z):    
    with open('511145_2.fasta', 'r') as genome: #читаем геном, так чтобы номера позиций соответствовали координатам, а не координатам -1
        line = genome.readlines()
        line1 = line[1:]
        line2 = ['R']
        for i in line1:
            i = i.replace('\n','')
            line2.append(i)
        genome_seq = "".join(line2)
        anti_genome_seq = opposite(genome_seq)
    
    with open('coordinate.txt', 'r') as coord:
        with open(y, 'r') as cover: #читаем файл с покрытием. Опять же, чтобы все соответствовало координатам
            cover_genome = ['cover']
            cover_file = cover.readlines()
            cover_file = cover_file[2:]
            cover_list = []
            for i in cover_file:
                i = i.replace('\n', '')
                cover_list.append(i)
            cover_genome += cover_list #в переменной cover_genome содержится покрытие для каждой позиции, причем позиция соответствует номеру в списке
            gene = coord.readlines()
        with open(z, 'r') as minus:
            cover_genome_anti = ['cover']
            cover_file_min = minus.readlines()
            cover_list_min = []
            for i in cover_file_min:
                i = i.replace('\n', '')
                cover_list_min.append(i)
            cover_genome_anti += cover_list_min #в переменной cover_genome содержится покрытие для каждой позиции, причем позиция соответствует номеру в списке
            gene_anti = coord.readlines()
            m=0       
        for line in gene: #проходимся по файлу с координатами. В зависимости от цепи применяем разную функцию для гексамеров.
            line = line.replace('\n','')
            line = line.split()
            start, end = int(line[0]), int(line[1])
            if line[2] == '+':
                cover_gene = cover_genome[start:end]
                seq_gene = genome_seq[start:end]
                cover_gene = list(map(float, cover_gene))
                s = sum(cover_gene)/(end - start)
            else:
                cover_gene_min = cover_genome_anti[start:end]
                seq_gene = anti_genome_seq[start:end]
                cover_gene_min = list(map(float, cover_gene))
                s = sum(cover_gene_min)/(end - start)
            hexa(seq_gene, s, cover_gene, str(line[2]))

Запуск подсчета покрытия гексомеров для трех реплик

In [9]:
programm('plus2.wig', 'minus2.wig')
programm('plus.wig', 'minus.wig')
programm('plus1.wig', 'minus1.wig')

Запись покрытия гексомеров в файл

In [ ]:
with open('hexa_all2.txt', 'w') as hex2:
    for hexa in hexa_dict:
        hex2.write(str(hexa) + ' ' + str(sum(hexa_dict[hexa])/len(hexa_dict[hexa])) + '\n')